In [1]:
import requests
from bs4 import BeautifulSoup
from math import ceil, nan
import json, re, os

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    page = BeautifulSoup(req.text,"html.parser")
    return page

def get_max(page):
    total = page.find("span",class_="plp-food-view__count").get_text()
    total = int("".join(re.findall("\d+",total)))
    return ceil(total/24)

def get_links(browser):
    # check for the xpath
    try:
        links = []
        xpath = '//*[@id="app"]/div/main/div[2]/div[1]/div[2]/div[2]/div[2]/div'
        WebDriverWait(browser,10).until(
            EC.presence_of_element_located((By.XPATH,xpath))
        )

        # scroll to the bottom
        actions = ActionChains(browser)
        element = browser.find_element(By.XPATH,xpath)
        actions.move_to_element(element).perform()

        # scrape all jobs
        page = BeautifulSoup(browser.page_source,"html.parser")
        ref = "https://www.carrefour.es"
        for a in page.find_all("a",class_="product-card__title-link"):
            link = a['href']
            if ref not in link:
                link = ref + link
            links.append(link)
        
        return links
    except: pass

In [27]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Chrome
import chromedriver_binary

try:
    browser.quit()
    browser = Chrome()
except:
    browser = Chrome()

url = "https://www.carrefour.es/supermercado/mascotas-promocion/F-105jZp643/c"
browser.get(url)

In [31]:
page = BeautifulSoup(browser.page_source,"html.parser")

In [23]:
# url = "https://www.carrefour.es/supermercado/mascotas-promocion/F-105jZp643/c"
# page = render_html(url)

In [ ]:
urls = [
    "https://www.carrefour.es/supermercado/mascotas-promocion/F-105jZp643/c",
    "https://www.carrefour.es/supermercado/el-mercado/cat20002/c",
    "https://www.carrefour.es/supermercado/bebidas/cat20003/c",
    "https://www.carrefour.es/supermercado/la-despensa/cat20001/c"
]

In [13]:
max_page = get_max(page)

6

In [ ]:
ref = "https://www.carrefour.es"
for a in page.find_all("a",class_="product-card__title-link"):
    link = a['href']
    if ref not in link:
        link = ref + link
    links.append(link)
    print(link)

24

In [53]:
from selenium.webdriver.common.action_chains import ActionChains

xpath = '//*[@id="app"]/div/main/div[2]/div[1]/div[2]/div[2]/div[2]/div/div[2]'

element = browser.find_element(By.CLASS_NAME, "pagination")

actions = ActionChains(browser)
actions.move_to_element(element).perform()

In [ ]:
# visit the url
url = "https://www.carrefour.es/supermercado/la-despensa/cat20001/c"
print(url)
browser.get(url)

# find the max. page
page = BeautifulSoup(browser.page_source,"html.parser")
max_page = get_max(page)

# find all product links
links = get_links(browser)

# iterate the next page
for i in range(1,max_page+1):
    new_url = url + f"?offset={i*24}"
    browser.get(new_url)
    links.extend(get_links(browser))
    print(f"{new_url} ({len(links)})")


In [61]:
links = list(dict.fromkeys(links))
json.dump(links,open("./urls/chips.json","w"),indent=4)
print("total products:",len(links))

total products: 949


In [2]:
root = "./urls/"
data = []
for file in os.listdir(root):
    if file.endswith("json"):
        data.extend(json.load(open(root+file)))
data = list(dict.fromkeys(data))
json.dump(data,open(root+"product_urls.json","w"),indent=4)
print(f"done. total urls {len(data)}.")

done. total urls 3325.
